# Handle Missing/Zero Values in Company Stock Data

This notebook handles missing values (represented as zeros) in company stock data files from the `4_separate_companies/merge_columns` folder.

**Note**: The data files use **zeros (0)** to represent missing values instead of NULL/NaN values.

## Process:
1. Load each Excel file
2. Analyze zero values for each company sheet
3. Decide strategy: remove rows or fill with appropriate values
4. Apply the strategy and save cleaned files

## Strategy:
- **Price columns (OPEN, HIGH, LOW, CLOSE)**: 
  - Replace zeros with NaN
  - Fill with forward fill then backward fill
- **Volume**: 
  - Replace zeros with NaN
  - Fill with median (more robust than mean for volume data)
- **Date columns**: No missing dates expected
- **Rows with all price data as zeros**: Remove those rows

## 1. Import Required Libraries

In [1]:
import pandas as pd
import numpy as np
import os
from openpyxl import load_workbook
import warnings
from datetime import datetime

warnings.filterwarnings('ignore')

print("Libraries imported successfully!")
print(f"Current date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Libraries imported successfully!
Current date: 2025-10-28 17:50:55


## 2. Configuration

In [2]:
# Input folder
input_folder = r'e:\Codes\Python\indexfundproject\work\4_separate_companies\merge_columns'

# Output folder (will be created if doesn't exist)
output_folder = r'e:\Codes\Python\indexfundproject\work\6_handle_missing_values\cleaned_data'

# Create output folder
os.makedirs(output_folder, exist_ok=True)

# Files to process
files_to_process = [
    'kse100_daily_data_sep_companies.xlsx',
    'kse30_daily_data_sep_companies.xlsx'
]

print(f"Input folder: {input_folder}")
print(f"Output folder: {output_folder}")
print(f"Files to process: {files_to_process}")

Input folder: e:\Codes\Python\indexfundproject\work\4_separate_companies\merge_columns
Output folder: e:\Codes\Python\indexfundproject\work\6_handle_missing_values\cleaned_data
Files to process: ['kse100_daily_data_sep_companies.xlsx', 'kse30_daily_data_sep_companies.xlsx']


## 3. Helper Functions

In [3]:
def analyze_zero_values(df, sheet_name):
    """
    Analyze zero values in numeric columns (treating zeros as missing values)
    Returns a summary of zero values
    """
    total_rows = len(df)
    zero_info = []
    
    # Get numeric columns only
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    
    for col in numeric_cols:
        zero_count = (df[col] == 0).sum()
        if zero_count > 0:
            zero_pct = (zero_count / total_rows) * 100
            zero_info.append({
                'Sheet': sheet_name,
                'Column': col,
                'Zero_Count': zero_count,
                'Zero_Percent': zero_pct,
                'Total_Rows': total_rows
            })
    
    return zero_info


def get_price_columns(df):
    """
    Identify price-related columns
    """
    price_keywords = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'PRICE']
    price_cols = [col for col in df.columns if any(kw in col.upper() for kw in price_keywords)]
    return price_cols


def get_volume_columns(df):
    """
    Identify volume-related columns
    """
    volume_cols = [col for col in df.columns if 'VOLUME' in col.upper()]
    return volume_cols


def handle_zero_values(df, sheet_name, strategy='smart'):
    """
    Handle zero values (treating them as missing values) based on column type
    
    Strategy:
    - Price columns: Replace zeros with NaN, then forward fill, then backward fill
    - Volume: Replace zeros with NaN, then fill with median
    - If entire row has all price data as zero: remove the row
    """
    df_clean = df.copy()
    actions_taken = []
    
    # Get column types
    price_cols = get_price_columns(df_clean)
    volume_cols = get_volume_columns(df_clean)
    
    # Check for rows where ALL price columns are zero
    if price_cols:
        all_price_zero = (df_clean[price_cols] == 0).all(axis=1)
        rows_to_remove = all_price_zero.sum()
        
        if rows_to_remove > 0:
            df_clean = df_clean[~all_price_zero]
            actions_taken.append(f"Removed {rows_to_remove} rows with all price data as zero")
    
    # Handle price columns - replace zeros with NaN, then forward fill, then backward fill
    for col in price_cols:
        if (df_clean[col] == 0).sum() > 0:
            zeros_before = (df_clean[col] == 0).sum()
            # Replace zeros with NaN
            df_clean[col] = df_clean[col].replace(0, np.nan)
            # Forward fill then backward fill
            df_clean[col] = df_clean[col].fillna(method='ffill').fillna(method='bfill')
            zeros_after = (df_clean[col] == 0).sum()
            filled = zeros_before - zeros_after
            if filled > 0:
                actions_taken.append(f"{col}: Replaced {filled} zeros using forward/backward fill")
    
    # Handle volume columns - replace zeros with median
    for col in volume_cols:
        if (df_clean[col] == 0).sum() > 0:
            zeros_before = (df_clean[col] == 0).sum()
            # Calculate median excluding zeros
            non_zero_values = df_clean[df_clean[col] != 0][col]
            if len(non_zero_values) > 0:
                median_value = non_zero_values.median()
                # Replace zeros with median
                df_clean[col] = df_clean[col].replace(0, median_value)
                zeros_after = (df_clean[col] == 0).sum()
                filled = zeros_before - zeros_after
                if filled > 0:
                    actions_taken.append(f"{col}: Replaced {filled} zeros with median ({median_value:,.0f})")
            else:
                # If all values are zero, just keep them
                actions_taken.append(f"{col}: All values are zero, kept as-is")
    
    # Handle any remaining numeric columns with median (excluding date-related columns)
    other_numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
    other_numeric_cols = [col for col in other_numeric_cols 
                         if col not in price_cols 
                         and col not in volume_cols
                         and 'DATE' not in col.upper()
                         and 'YEAR' not in col.upper()
                         and 'MONTH' not in col.upper()
                         and 'DAY' not in col.upper()]
    
    for col in other_numeric_cols:
        if (df_clean[col] == 0).sum() > 0:
            zeros_before = (df_clean[col] == 0).sum()
            # Calculate median excluding zeros
            non_zero_values = df_clean[df_clean[col] != 0][col]
            if len(non_zero_values) > 0:
                median_value = non_zero_values.median()
                # Replace zeros with median
                df_clean[col] = df_clean[col].replace(0, median_value)
                zeros_after = (df_clean[col] == 0).sum()
                filled = zeros_before - zeros_after
                if filled > 0:
                    actions_taken.append(f"{col}: Replaced {filled} zeros with median ({median_value:.2f})")
    
    return df_clean, actions_taken


print("Helper functions defined successfully!")

Helper functions defined successfully!


## 4. Analyze Zero Values in All Files

First, let's scan all files and sheets to see the extent of zero values (missing data)

In [4]:
print("="*100)
print("PHASE 1: ANALYZING ZERO VALUES (MISSING DATA)")
print("="*100)

all_zero_info = []
file_summaries = {}

for filename in files_to_process:
    filepath = os.path.join(input_folder, filename)
    
    if not os.path.exists(filepath):
        print(f"\n⚠️ File not found: {filename}")
        continue
    
    print(f"\n{'='*100}")
    print(f"📁 Processing: {filename}")
    print(f"{'='*100}")
    
    # Load Excel file
    xl = pd.ExcelFile(filepath)
    sheet_names = xl.sheet_names
    
    print(f"Total sheets: {len(sheet_names)}")
    
    sheets_with_zeros = 0
    total_zero_values = 0
    
    for idx, sheet_name in enumerate(sheet_names, 1):
        # Read sheet
        df = pd.read_excel(filepath, sheet_name=sheet_name)
        
        # Analyze zero values
        zero_info = analyze_zero_values(df, sheet_name)
        
        if zero_info:
            sheets_with_zeros += 1
            all_zero_info.extend(zero_info)
            
            # Calculate total zeros for this sheet
            sheet_zero_count = sum([info['Zero_Count'] for info in zero_info])
            total_zero_values += sheet_zero_count
            
            if idx <= 5 or sheets_with_zeros <= 10:  # Show details for first few sheets
                print(f"\n  📊 Sheet: {sheet_name}")
                print(f"     Rows: {len(df)}, Columns: {len(df.columns)}")
                print(f"     Zero values found:")
                for info in zero_info:
                    print(f"       - {info['Column']}: {info['Zero_Count']} ({info['Zero_Percent']:.2f}%)")
        
        # Progress indicator
        if idx % 50 == 0:
            print(f"  ... Processed {idx}/{len(sheet_names)} sheets")
    
    file_summaries[filename] = {
        'total_sheets': len(sheet_names),
        'sheets_with_zeros': sheets_with_zeros,
        'total_zero_values': total_zero_values
    }
    
    print(f"\n  ✓ Summary: {sheets_with_zeros}/{len(sheet_names)} sheets have zero values")
    print(f"  ✓ Total zero data points: {total_zero_values:,}")

print(f"\n{'='*100}")
print("OVERALL SUMMARY")
print(f"{'='*100}")
for filename, summary in file_summaries.items():
    print(f"\n{filename}:")
    print(f"  Total sheets: {summary['total_sheets']}")
    print(f"  Sheets with zero values: {summary['sheets_with_zeros']}")
    print(f"  Total zero data points: {summary['total_zero_values']:,}")

PHASE 1: ANALYZING ZERO VALUES (MISSING DATA)

📁 Processing: kse100_daily_data_sep_companies.xlsx


Total sheets: 141

  📊 Sheet: ABL
     Rows: 1411, Columns: 11
     Zero values found:
       - IDX WT %: 4 (0.28%)
       - VOLUME: 45 (3.19%)

  📊 Sheet: ABL
     Rows: 1411, Columns: 11
     Zero values found:
       - IDX WT %: 4 (0.28%)
       - VOLUME: 45 (3.19%)

  📊 Sheet: ABOT
     Rows: 1411, Columns: 11
     Zero values found:
       - IDX WT %: 4 (0.28%)
       - VOLUME: 18 (1.28%)

  📊 Sheet: ABOT
     Rows: 1411, Columns: 11
     Zero values found:
       - IDX WT %: 4 (0.28%)
       - VOLUME: 18 (1.28%)

  📊 Sheet: AGIL
     Rows: 429, Columns: 11
     Zero values found:
       - VOLUME: 49 (11.42%)

  📊 Sheet: AGIL
     Rows: 429, Columns: 11
     Zero values found:
       - VOLUME: 49 (11.42%)

  📊 Sheet: AGP
     Rows: 1411, Columns: 11
     Zero values found:
       - IDX WT %: 4 (0.28%)
       - VOLUME: 1 (0.07%)

  📊 Sheet: AGP
     Rows: 1411, Columns: 11
     Zero values found:
       - IDX WT %: 4 (0.28%)
       - VOLUME: 1 (0.07%)

  📊 Sheet: AICL
     Rows: 14

## 5. Create Zero Values Summary DataFrame

In [5]:
if all_zero_info:
    zero_df = pd.DataFrame(all_zero_info)
    
    print(f"Total records with zero values: {len(zero_df)}")
    print(f"\nTop 20 sheets with most zero values:")
    
    # Group by sheet and sum zero counts
    sheet_summary = zero_df.groupby('Sheet').agg({
        'Zero_Count': 'sum',
        'Total_Rows': 'first'
    }).sort_values('Zero_Count', ascending=False).head(20)
    
    sheet_summary['Zero_Percent'] = (sheet_summary['Zero_Count'] / sheet_summary['Total_Rows']) * 100
    display(sheet_summary)
    
    print(f"\nZero values by column type:")
    column_summary = zero_df.groupby('Column')['Zero_Count'].sum().sort_values(ascending=False)
    display(column_summary.head(10))
else:
    print("✓ No zero values found in any numeric columns!")

Total records with zero values: 202

Top 20 sheets with most zero values:


,Zero_Count,Total_Rows,Zero_Percent
Sheet,,,
PSEL,781,1411,55.350815
IBFL,547,1285,42.568093
PGLC,478,858,55.710956
EFUG,396,1410,28.085106
POML,351,1129,31.089460
PAKT,299,1411,21.190645
JDWS,295,925,31.891892
BNWM,283,1411,20.056697
SML,259,369,70.189702



Zero values by column type:


Column
VOLUME      6305
IDX WT %     791
Name: Zero_Count, dtype: int64

## 6. Process Files and Handle Zero Values

Now let's apply our strategy to handle zero values and save cleaned files

In [6]:
print("="*100)
print("PHASE 2: HANDLING ZERO VALUES AND SAVING CLEANED FILES")
print("="*100)

processing_log = []

for filename in files_to_process:
    filepath = os.path.join(input_folder, filename)
    
    if not os.path.exists(filepath):
        print(f"\n⚠️ File not found: {filename}")
        continue
    
    print(f"\n{'='*100}")
    print(f"📁 Processing: {filename}")
    print(f"{'='*100}")
    
    # Load Excel file
    xl = pd.ExcelFile(filepath)
    sheet_names = xl.sheet_names
    
    # Output file path
    output_filepath = os.path.join(output_folder, filename)
    
    # Create Excel writer
    with pd.ExcelWriter(output_filepath, engine='openpyxl') as writer:
        
        sheets_processed = 0
        sheets_cleaned = 0
        total_rows_removed = 0
        total_zeros_replaced = 0
        
        for idx, sheet_name in enumerate(sheet_names, 1):
            # Read sheet
            df = pd.read_excel(filepath, sheet_name=sheet_name)
            original_rows = len(df)
            
            # Count zeros in numeric columns
            numeric_cols = df.select_dtypes(include=[np.number]).columns
            original_zeros = sum((df[col] == 0).sum() for col in numeric_cols)
            
            # Handle zero values
            df_clean, actions = handle_zero_values(df, sheet_name)
            
            # Calculate changes
            final_rows = len(df_clean)
            numeric_cols_clean = df_clean.select_dtypes(include=[np.number]).columns
            final_zeros = sum((df_clean[col] == 0).sum() for col in numeric_cols_clean)
            rows_removed = original_rows - final_rows
            zeros_replaced = original_zeros - final_zeros
            
            if actions:
                sheets_cleaned += 1
                total_rows_removed += rows_removed
                total_zeros_replaced += zeros_replaced
                
                # Log actions for first few sheets
                if sheets_cleaned <= 10:
                    print(f"\n  ✓ Sheet: {sheet_name}")
                    for action in actions:
                        print(f"     - {action}")
                
                # Store in log
                processing_log.append({
                    'File': filename,
                    'Sheet': sheet_name,
                    'Original_Rows': original_rows,
                    'Final_Rows': final_rows,
                    'Rows_Removed': rows_removed,
                    'Original_Zeros': original_zeros,
                    'Final_Zeros': final_zeros,
                    'Zeros_Replaced': zeros_replaced,
                    'Actions': '; '.join(actions)
                })
            
            # Write to Excel
            df_clean.to_excel(writer, sheet_name=sheet_name, index=False)
            sheets_processed += 1
            
            # Progress indicator
            if idx % 50 == 0:
                print(f"  ... Processed {idx}/{len(sheet_names)} sheets")
        
        print(f"\n  ✓ File processed: {sheets_processed} sheets")
        print(f"  ✓ Sheets cleaned: {sheets_cleaned}")
        print(f"  ✓ Total rows removed: {total_rows_removed:,}")
        print(f"  ✓ Total zeros replaced: {total_zeros_replaced:,}")
    
    print(f"  ✓ Saved to: {output_filepath}")

print(f"\n{'='*100}")
print("✅ ALL FILES PROCESSED SUCCESSFULLY!")
print(f"{'='*100}")
print(f"Output folder: {output_folder}")

PHASE 2: HANDLING ZERO VALUES AND SAVING CLEANED FILES

📁 Processing: kse100_daily_data_sep_companies.xlsx

PHASE 2: HANDLING ZERO VALUES AND SAVING CLEANED FILES

📁 Processing: kse100_daily_data_sep_companies.xlsx

  ✓ Sheet: ABL
     - VOLUME: Replaced 45 zeros with median (28,370)
     - IDX WT %: Replaced 4 zeros with median (0.50)

  ✓ Sheet: ABL
     - VOLUME: Replaced 45 zeros with median (28,370)
     - IDX WT %: Replaced 4 zeros with median (0.50)

  ✓ Sheet: ABOT
     - VOLUME: Replaced 18 zeros with median (8,179)
     - IDX WT %: Replaced 4 zeros with median (0.67)

  ✓ Sheet: ABOT
     - VOLUME: Replaced 18 zeros with median (8,179)
     - IDX WT %: Replaced 4 zeros with median (0.67)

  ✓ Sheet: AGIL
     - VOLUME: Replaced 49 zeros with median (5,050)

  ✓ Sheet: AGIL
     - VOLUME: Replaced 49 zeros with median (5,050)

  ✓ Sheet: AGP
     - VOLUME: Replaced 1 zeros with median (119,435)
     - IDX WT %: Replaced 4 zeros with median (0.41)

  ✓ Sheet: AGP
     - VOLUME:

## 7. Processing Log Summary

In [7]:
if processing_log:
    log_df = pd.DataFrame(processing_log)
    
    print("Processing Log Summary:")
    print("="*100)
    
    # Overall statistics
    print(f"\nTotal sheets cleaned: {len(log_df)}")
    print(f"Total rows removed: {log_df['Rows_Removed'].sum():,}")
    print(f"Total zeros replaced: {log_df['Zeros_Replaced'].sum():,}")
    print(f"Total zero values resolved: {(log_df['Original_Zeros'] - log_df['Final_Zeros']).sum():,}")
    
    # Show sheets with most changes
    print(f"\nTop 10 sheets with most zeros replaced:")
    display(log_df.nlargest(10, 'Zeros_Replaced')[['File', 'Sheet', 'Original_Zeros', 'Zeros_Replaced', 'Final_Zeros']])
    
    print(f"\nTop 10 sheets with most rows removed:")
    display(log_df.nlargest(10, 'Rows_Removed')[['File', 'Sheet', 'Original_Rows', 'Rows_Removed', 'Final_Rows']])
    
    # Save log to CSV
    log_filepath = os.path.join(output_folder, 'processing_log.csv')
    log_df.to_csv(log_filepath, index=False)
    print(f"\n✓ Processing log saved to: {log_filepath}")
    
    # Display full log
    print(f"\nFull Processing Log (first 20 entries):")
    display(log_df.head(20))
else:
    print("No zero values were found or processed.")

Processing Log Summary:

Total sheets cleaned: 149
Total rows removed: 0
Total zeros replaced: 7,096
Total zero values resolved: 7,096

Top 10 sheets with most zeros replaced:


,File,Sheet,Original_Zeros,Zeros_Replaced,Final_Zeros
95,kse100_daily_data_sep_companies.xlsx,PSEL,781,781,0
49,kse100_daily_data_sep_companies.xlsx,IBFL,547,547,0
86,kse100_daily_data_sep_companies.xlsx,PGLC,478,478,0
26,kse100_daily_data_sep_companies.xlsx,EFUG,396,396,0
93,kse100_daily_data_sep_companies.xlsx,POML,351,351,0
85,kse100_daily_data_sep_companies.xlsx,PAKT,299,299,0
57,kse100_daily_data_sep_companies.xlsx,JDWS,295,295,0
15,kse100_daily_data_sep_companies.xlsx,BNWM,283,283,0
105,kse100_daily_data_sep_companies.xlsx,SML,259,259,0
37,kse100_daily_data_sep_companies.xlsx,GADT,246,246,0



Top 10 sheets with most rows removed:


,File,Sheet,Original_Rows,Rows_Removed,Final_Rows
0,kse100_daily_data_sep_companies.xlsx,ABL,1411,0,1411
1,kse100_daily_data_sep_companies.xlsx,ABOT,1411,0,1411
2,kse100_daily_data_sep_companies.xlsx,AGIL,429,0,429
3,kse100_daily_data_sep_companies.xlsx,AGP,1411,0,1411
4,kse100_daily_data_sep_companies.xlsx,AICL,1411,0,1411
5,kse100_daily_data_sep_companies.xlsx,AKBL,1411,0,1411
6,kse100_daily_data_sep_companies.xlsx,ANL,672,0,672
7,kse100_daily_data_sep_companies.xlsx,APL,1411,0,1411
8,kse100_daily_data_sep_companies.xlsx,ARPL,1043,0,1043
9,kse100_daily_data_sep_companies.xlsx,ASL,243,0,243



✓ Processing log saved to: e:\Codes\Python\indexfundproject\work\6_handle_missing_values\cleaned_data\processing_log.csv

Full Processing Log (first 20 entries):


,File,Sheet,Original_Rows,Final_Rows,Rows_Removed,Original_Zeros,Final_Zeros,Zeros_Replaced,Actions
0,kse100_daily_data_sep_companies.xlsx,ABL,1411,1411,0,49,0,49,"VOLUME: Replaced 45 zeros with median (28,370)..."
1,kse100_daily_data_sep_companies.xlsx,ABOT,1411,1411,0,22,0,22,"VOLUME: Replaced 18 zeros with median (8,179);..."
2,kse100_daily_data_sep_companies.xlsx,AGIL,429,429,0,49,0,49,"VOLUME: Replaced 49 zeros with median (5,050)"
3,kse100_daily_data_sep_companies.xlsx,AGP,1411,1411,0,5,0,5,"VOLUME: Replaced 1 zeros with median (119,435)..."
4,kse100_daily_data_sep_companies.xlsx,AICL,1411,1411,0,21,0,21,"VOLUME: Replaced 17 zeros with median (194,750..."
5,kse100_daily_data_sep_companies.xlsx,AKBL,1411,1411,0,6,0,6,"VOLUME: Replaced 2 zeros with median (309,500)..."
6,kse100_daily_data_sep_companies.xlsx,ANL,672,672,0,4,0,4,IDX WT %: Replaced 4 zeros with median (0.32)
7,kse100_daily_data_sep_companies.xlsx,APL,1411,1411,0,5,0,5,"VOLUME: Replaced 1 zeros with median (32,393);..."
8,kse100_daily_data_sep_companies.xlsx,ARPL,1043,1043,0,68,0,68,"VOLUME: Replaced 64 zeros with median (2,050);..."
9,kse100_daily_data_sep_companies.xlsx,ASL,243,243,0,4,0,4,IDX WT %: Replaced 4 zeros with median (0.26)


## 8. Verification - Sample Check

Let's verify a few sheets from the cleaned files to ensure quality

In [8]:
print("="*100)
print("VERIFICATION: Checking cleaned files for remaining zero values")
print("="*100)

verification_results = []

for filename in files_to_process:
    output_filepath = os.path.join(output_folder, filename)
    
    if not os.path.exists(output_filepath):
        continue
    
    print(f"\n📁 Verifying: {filename}")
    
    xl = pd.ExcelFile(output_filepath)
    sheet_names = xl.sheet_names
    
    sheets_with_zeros = 0
    total_zeros = 0
    
    for sheet_name in sheet_names:
        df = pd.read_excel(output_filepath, sheet_name=sheet_name)
        
        # Count zeros in numeric columns (excluding potential ID or index columns)
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        zero_count = sum((df[col] == 0).sum() for col in numeric_cols)
        
        if zero_count > 0:
            sheets_with_zeros += 1
            total_zeros += zero_count
            
            verification_results.append({
                'File': filename,
                'Sheet': sheet_name,
                'Remaining_Zeros': zero_count
            })
    
    print(f"  Total sheets: {len(sheet_names)}")
    print(f"  Sheets with remaining zero values: {sheets_with_zeros}")
    print(f"  Total remaining zero values: {total_zeros:,}")
    
    if sheets_with_zeros == 0:
        print(f"  ✅ All sheets have no problematic zeros!")
    else:
        print(f"  ⚠️ Some sheets still have zero values (may be legitimate)")

if verification_results:
    print(f"\n⚠️ Sheets with remaining zero values:")
    verify_df = pd.DataFrame(verification_results)
    display(verify_df.head(20))
    print(f"\nNote: Some zeros may be legitimate (e.g., no trading occurred)")
else:
    print(f"\n✅ ALL FILES PROCESSED! No zero values remaining.")

VERIFICATION: Checking cleaned files for remaining zero values

📁 Verifying: kse100_daily_data_sep_companies.xlsx

VERIFICATION: Checking cleaned files for remaining zero values

📁 Verifying: kse100_daily_data_sep_companies.xlsx
  Total sheets: 141
  Sheets with remaining zero values: 0
  Total remaining zero values: 0
  ✅ All sheets have no problematic zeros!

📁 Verifying: kse30_daily_data_sep_companies.xlsx
  Total sheets: 141
  Sheets with remaining zero values: 0
  Total remaining zero values: 0
  ✅ All sheets have no problematic zeros!

📁 Verifying: kse30_daily_data_sep_companies.xlsx
  Total sheets: 59
  Sheets with remaining zero values: 0
  Total remaining zero values: 0
  ✅ All sheets have no problematic zeros!

✅ ALL FILES PROCESSED! No zero values remaining.
  Total sheets: 59
  Sheets with remaining zero values: 0
  Total remaining zero values: 0
  ✅ All sheets have no problematic zeros!

✅ ALL FILES PROCESSED! No zero values remaining.


## 9. Sample Data Comparison

Compare a sample sheet before and after cleaning

In [9]:
# Pick a sample file and sheet for comparison
if processing_log:
    # Get a sheet that had zero values
    sample_record = processing_log[0]  # First cleaned sheet
    sample_file = sample_record['File']
    sample_sheet = sample_record['Sheet']
    
    print(f"Sample Comparison: {sample_file} - Sheet: {sample_sheet}")
    print("="*100)
    
    # Load original
    original_path = os.path.join(input_folder, sample_file)
    df_original = pd.read_excel(original_path, sheet_name=sample_sheet)
    
    # Load cleaned
    cleaned_path = os.path.join(output_folder, sample_file)
    df_cleaned = pd.read_excel(cleaned_path, sheet_name=sample_sheet)
    
    print(f"\nOriginal Data:")
    print(f"  Rows: {len(df_original)}")
    numeric_cols_orig = df_original.select_dtypes(include=[np.number]).columns
    total_zeros_orig = sum((df_original[col] == 0).sum() for col in numeric_cols_orig)
    print(f"  Zero values in numeric columns: {total_zeros_orig}")
    print(f"  Zeros by column:")
    for col in numeric_cols_orig:
        zeros = (df_original[col] == 0).sum()
        if zeros > 0:
            print(f"    {col}: {zeros}")
    
    print(f"\nCleaned Data:")
    print(f"  Rows: {len(df_cleaned)}")
    numeric_cols_clean = df_cleaned.select_dtypes(include=[np.number]).columns
    total_zeros_clean = sum((df_cleaned[col] == 0).sum() for col in numeric_cols_clean)
    print(f"  Zero values in numeric columns: {total_zeros_clean}")
    print(f"  Zeros by column:")
    for col in numeric_cols_clean:
        zeros = (df_cleaned[col] == 0).sum()
        if zeros > 0:
            print(f"    {col}: {zeros}")
    
    print(f"\n  Changes: Removed {len(df_original) - len(df_cleaned)} rows, Replaced {total_zeros_orig - total_zeros_clean} zeros")
    
    print(f"\nFirst 5 rows of original data:")
    display(df_original.head())
    
    print(f"\nFirst 5 rows of cleaned data:")
    display(df_cleaned.head())
    
    # Show some rows that had zeros
    print(f"\nSample rows from original that had zeros (if any):")
    price_cols = get_price_columns(df_original)
    if price_cols:
        rows_with_zeros = df_original[(df_original[price_cols] == 0).any(axis=1)].head(3)
        if len(rows_with_zeros) > 0:
            display(rows_with_zeros)
        else:
            print("  No rows with zero price values in the sample")
else:
    print("No data to compare - no zero values were processed.")

Sample Comparison: kse100_daily_data_sep_companies.xlsx - Sheet: ABL

Original Data:
  Rows: 1411
  Zero values in numeric columns: 49
  Zeros by column:
    IDX WT %: 4
    VOLUME: 45

Cleaned Data:
  Rows: 1411
  Zero values in numeric columns: 0
  Zeros by column:

  Changes: Removed 0 rows, Replaced 49 zeros

First 5 rows of original data:

Original Data:
  Rows: 1411
  Zero values in numeric columns: 49
  Zeros by column:
    IDX WT %: 4
    VOLUME: 45

Cleaned Data:
  Rows: 1411
  Zero values in numeric columns: 0
  Zeros by column:

  Changes: Removed 0 rows, Replaced 49 zeros

First 5 rows of original data:


,Date,ISIN,SYMBOL,COMPANY,PRICE,IDX WT %,FF BASED SHARES,FF BASED MCAP,ORD SHARES,ORD SHARES MCAP,VOLUME
0,2020-01-01,PK0083501012,ABL,Allied Bank Ltd.,96.98,0.9,171761075,16657389053.5,1145073830,111049260033.4,98000
1,2020-01-02,PK0083501012,ABL,Allied Bank Ltd.,100.01,0.9,171761075,17177825110.75,1145073830,114518833738.3,253500
2,2020-01-03,PK0083501012,ABL,Allied Bank Ltd.,100.10,0.9,171761075,17193283607.5,1145073830,114621890383.0,159500
3,2020-01-06,PK0083501012,ABL,Allied Bank Ltd.,97.63,0.9,171761075,16769033752.25,1145073830,111793558022.9,63000
4,2020-01-07,PK0083501012,ABL,Allied Bank Ltd.,98.05,0.9,171761075,16841173403.75,1145073830,112274489031.5,57000



First 5 rows of cleaned data:


,Date,ISIN,SYMBOL,COMPANY,PRICE,IDX WT %,FF BASED SHARES,FF BASED MCAP,ORD SHARES,ORD SHARES MCAP,VOLUME
0,2020-01-01,PK0083501012,ABL,Allied Bank Ltd.,96.98,0.9,171761075,16657389053.5,1145073830,111049260033.4,98000
1,2020-01-02,PK0083501012,ABL,Allied Bank Ltd.,100.01,0.9,171761075,17177825110.75,1145073830,114518833738.3,253500
2,2020-01-03,PK0083501012,ABL,Allied Bank Ltd.,100.10,0.9,171761075,17193283607.5,1145073830,114621890383.0,159500
3,2020-01-06,PK0083501012,ABL,Allied Bank Ltd.,97.63,0.9,171761075,16769033752.25,1145073830,111793558022.9,63000
4,2020-01-07,PK0083501012,ABL,Allied Bank Ltd.,98.05,0.9,171761075,16841173403.75,1145073830,112274489031.5,57000



Sample rows from original that had zeros (if any):
  No rows with zero price values in the sample


## 10. Final Summary Report

In [10]:
print("="*100)
print("FINAL SUMMARY REPORT")
print("="*100)

print(f"\n📂 Input Folder: {input_folder}")
print(f"📂 Output Folder: {output_folder}")

print(f"\n📊 Files Processed:")
for filename in files_to_process:
    input_path = os.path.join(input_folder, filename)
    output_path = os.path.join(output_folder, filename)
    
    if os.path.exists(output_path):
        print(f"  ✓ {filename}")
    else:
        print(f"  ✗ {filename} - Not processed")

if processing_log:
    log_df = pd.DataFrame(processing_log)
    
    print(f"\n📈 Statistics:")
    print(f"  Total sheets processed: {len(log_df):,}")
    print(f"  Total original zero values: {log_df['Original_Zeros'].sum():,}")
    print(f"  Total rows removed: {log_df['Rows_Removed'].sum():,}")
    print(f"  Total zeros replaced: {log_df['Zeros_Replaced'].sum():,}")
    print(f"  Final remaining zeros: {log_df['Final_Zeros'].sum():,}")
    
    success_rate = ((log_df['Original_Zeros'].sum() - log_df['Final_Zeros'].sum()) / log_df['Original_Zeros'].sum() * 100) if log_df['Original_Zeros'].sum() > 0 else 100
    print(f"  Success rate: {success_rate:.2f}%")

print(f"\n🎯 Strategy Used:")
print(f"  - Price columns (OPEN, HIGH, LOW, CLOSE): Replace zeros with NaN → Forward fill → Backward fill")
print(f"  - Volume columns: Replace zeros with median (calculated from non-zero values)")
print(f"  - Rows with all price data as zero: Removed")
print(f"  - Other numeric columns: Replace zeros with median (calculated from non-zero values)")

print(f"\n✅ Processing complete!")
print(f"\nCleaned files saved to: {output_folder}")
print(f"Processing log saved to: {os.path.join(output_folder, 'processing_log.csv')}")
print("="*100)

FINAL SUMMARY REPORT

📂 Input Folder: e:\Codes\Python\indexfundproject\work\4_separate_companies\merge_columns
📂 Output Folder: e:\Codes\Python\indexfundproject\work\6_handle_missing_values\cleaned_data

📊 Files Processed:
  ✓ kse100_daily_data_sep_companies.xlsx
  ✓ kse30_daily_data_sep_companies.xlsx

📈 Statistics:
  Total sheets processed: 149
  Total original zero values: 7,096
  Total rows removed: 0
  Total zeros replaced: 7,096
  Final remaining zeros: 0
  Success rate: 100.00%

🎯 Strategy Used:
  - Price columns (OPEN, HIGH, LOW, CLOSE): Replace zeros with NaN → Forward fill → Backward fill
  - Volume columns: Replace zeros with median (calculated from non-zero values)
  - Rows with all price data as zero: Removed
  - Other numeric columns: Replace zeros with median (calculated from non-zero values)

✅ Processing complete!

Cleaned files saved to: e:\Codes\Python\indexfundproject\work\6_handle_missing_values\cleaned_data
Processing log saved to: e:\Codes\Python\indexfundproject